<a href="https://colab.research.google.com/github/harnalashok/hadoop/blob/main/spark_install_on_hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 27th March, 2023
# Myfolder: github/hadoop
# Objective:
#            i)  Install hadoop on colab
#                hadoop version is 3.3.5
#            ii) Experiments with hadoop
#           iii) Install spark on colab
#                 spark version is 3.3.2
#            iv) Access file on hadoop from spark
#
# ****************************************
#  Anytime hadoop is in safemode, execute:           
#  !hdfs dfsadmin -safemode leave
# ****************************************
#
# Java 8 install: https://stackoverflow.com/a/58191107
# Hadoop install: https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
# Spark install:  https://stackoverflow.com/a/64183749
#                 https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

## Install hadoop
Install and configure hadoop

### Define functions
No downloads. Just function definitions

In [1]:
# 1.0 How to set environment variable
import os  
import time  

#### ssh_install()

In [2]:
# 2.0 Function to install ssh client and sshd (Server)
def ssh_install():
  print("\n--1. Download and install ssh server----\n")
  ! sudo apt-get remove openssh-client openssh-server
  ! sudo apt install openssh-client openssh-server
  
  print("\n--2. Restart ssh server----\n")
  ! service ssh restart

#### Java install

In [3]:
# 3.0 Function to download and install java 8
def install_java():
  ! rm -rf /usr/java

  print("\n--Download and install Java 8----\n")
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null        # install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     # set environment variable

  !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  !update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
  
  !mkdir -p /usr/java
  ! ln -s "/usr/lib/jvm/java-8-openjdk-amd64"  "/usr/java"
  ! mv "/usr/java/java-8-openjdk-amd64"  "/usr/java/latest"
  
  !java -version       #check java version
  !javac -version

#### hadoop install

In [4]:
# 4.0 Function to download and install hadoop
def hadoop_install():
  print("\n--5. Download hadoop tar.gz----\n")
  ! wget -c https://dlcdn.apache.org/hadoop/common/hadoop-3.3.5/hadoop-3.3.5.tar.gz

  print("\n--6. Transfer downloaded content and unzip tar.gz----\n")
  !  mv /content/hadoop*   /opt/
  ! tar -xzf /opt/hadoop-3.3.5.tar.gz  --directory /opt/

  print("\n--7. Create hadoop folder----\n")
  ! rm -r /app/hadoop/tmp
  ! mkdir  -p   /app/hadoop/tmp
  
  print("\n--8. Check folder for files----\n")
  ! ls -la /opt

#### hadoop config

In [5]:
# 5.0 Function for setting hadoop configuration
def hadoop_config():
  print("\n--Begin Configuring hadoop---\n")
  print("\n=============================\n")
  print("\n--9. core-site.xml----\n")
  ! cat  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml

  print("\n--10. Amend core-site.xml----\n")
  !  echo  '<?xml version="1.0" encoding="UTF-8"?>' >  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  ' <configuration>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '    <property>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '        <name>fs.defaultFS</name>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '        <value>hdfs://localhost:9000</value>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '    </property>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '    <property>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '          <name>hadoop.tmp.dir</name>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '          <value>/app/hadoop/tmp</value>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '       <description>A base for other temporary directories.</description>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '     </property>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  # Added following regarding safemode from here:
  # https://stackoverflow.com/a/33800253
  !  echo  '     <property>'    >> /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '      <name>dfs.safemode.threshold.pct</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '    <value>0</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '  </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/core-site.xml
  !  echo  '  </configuration>' >>  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml

  print("\n--11. Amended core-site.xml----\n")
  ! cat  /opt/hadoop-3.3.5/etc/hadoop/core-site.xml

  print("\n--12. yarn-site.xml----\n")
  !cat /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml

  !echo '<?xml version="1.0" encoding="UTF-8"?>' > /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '<configuration>' >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '    <property>' >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '        <name>yarn.nodemanager.aux-services</name>' >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '        <value>mapreduce_shuffle</value>' >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '    </property>' >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '    <property>'  >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '       <name>yarn.nodemanager.vmem-check-enabled</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '       <value>false</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo '    </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  !echo ' </configuration>'  >> /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml
  
  print("\n--13. Amended yarn-site.xml----\n")
  !cat /opt/hadoop-3.3.5/etc/hadoop/yarn-site.xml

  print("\n--14. mapred-site.xml----\n")
  !cat  /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml

  print("\n--15. Amend mapred-site.xml----\n")
  !echo '<?xml version="1.0"?>'  > /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '<configuration>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '    <property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '       <name>mapreduce.framework.name</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '        <value>yarn</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '    </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '    <property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '        <name>yarn.app.mapreduce.am.env</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '        <value>HADOOP_MAPRED_HOME=${HADOOP_HOME}</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '    </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '    <property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '       <name>mapreduce.map.env</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '       <value>HADOOP_MAPRED_HOME=${HADOOP_HOME}</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '    </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '    <property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '      <name>mapreduce.reduce.env</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '      <value>HADOOP_MAPRED_HOME=${HADOOP_HOME}</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '   </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml
  !echo '</configuration>'  >> /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml

  print("\n--16, Amended mapred-site.xml----\n")
  !cat  /opt/hadoop-3.3.5/etc/hadoop/mapred-site.xml

  print("\n---17. hdfs-site.xml----\n")
  !cat  /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  
  print("\n---18. Amend hdfs-site.xml----\n")
  !echo  '<?xml version="1.0" encoding="UTF-8"?> '   > /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>' >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '<configuration>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '    <property>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '        <name>dfs.replication</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '        <value>1</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '    </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '   <property>'   >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '        <name>dfs.block.size</name>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '        <value>16777216</value>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '        <description>Block size</description>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '  </property>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml
  !echo  '</configuration>'  >> /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml

  print("\n---19. Amended hdfs-site.xml----\n")
  !cat  /opt/hadoop-3.3.5/etc/hadoop/hdfs-site.xml

  print("\n---20. hadoop-env.sh----\n")
  # https://stackoverflow.com/a/53140448
  !cat /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh
  ! echo 'export JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"' >> /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh
  ! echo 'export HDFS_NAMENODE_USER="root"'  >> /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh
  ! echo 'export HDFS_DATANODE_USER="root"'  >> /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh
  ! echo 'export HDFS_SECONDARYNAMENODE_USER="root"'  >> /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh
  ! echo 'export YARN_RESOURCEMANAGER_USER="root"'  >> /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh
  ! echo 'export YARN_NODEMANAGER_USER="root"'  >> /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh
  
  print("\n---21. Amended hadoop-env.sh----\n")
  !cat /opt/hadoop-3.3.5/etc/hadoop/hadoop-env.sh


#### ssh keys

In [6]:
# 6.0 Function tp setup ssh passphrase
def set_keys():
  print("\n---22.0 Deleting existing SSH keys----\n")
  ! rm -r -f /root/.ssh

  print("\n---22.1 Generate SSH keys----\n")
  ! cd ~ ; pwd 
  ! cd ~ ; ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
  ! cd ~ ; cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
  ! cd ~ ; chmod 0600 ~/.ssh/authorized_keys


#### Set environment

In [7]:
# 7.0 Function to set up environmental variables
def set_env():
  print("\n---23. Set Environment variables----\n")
  # 'export' command does not work in colab
  # https://stackoverflow.com/a/57240319
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"   
  os.environ["HADOOP_HOME"] = "/opt/hadoop-3.3.5"
  os.environ["HADOOP_CONF_DIR"] = "/opt/hadoop-3.3.5/etc/hadoop" 
  os.environ["LD_LIBRARY_PATH"] += ":/opt/hadoop-3.3.5/lib/native"
  os.environ["PATH"] += ":/opt/hadoop-3.3.5/bin:/opt/hadoop-3.3.5/sbin"

#### Install all function

In [8]:
# 8.0 Function to call all functions
def install_hadoop():
  print("\n--Install java----\n")
  ssh_install()
  install_java()  
  hadoop_install()
  hadoop_config()
  set_keys()
  set_env()


### Begin install
Start downloading, install and configure. Takes around 1 minute

In [ ]:
# 9.0 Start installation
start = time.time()
install_hadoop()
end = time.time()
print("\n---Time taken----\n")
print((end- start)/60)

### Format hadoop

In [ ]:
# 10.0 Format hadoop
print("\n---24. Format namenode----\n")
!hdfs namenode  -format

## Start hadoop
And create folders



>If namenode is in safemode, use the command:   
`!hdfs dfsadmin -safemode leave`


>If start fails with 'Connection refused', run once again  
 `ssh_install()` OR restart runtime `(ctrl+M)`

> Folders created are:
>> /user <br>
>> /user/ashok

>> A file `airports.csv` is also created and deposited in /user/ashok 

In [19]:
# 11.0 Start namenode
#      If this fails, run
#       ssh_install() below
#        and start hadoop again:

print("\n---25. Start namenode----\n")
! start-dfs.sh


---25. Start namenode----

Starting namenodes on [localhost]
localhost: namenode is running as process 39485.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.
Starting datanodes
Starting secondary namenodes [d89213ccffe7]
d89213ccffe7: secondarynamenode is running as process 5960.  Stop it first and ensure /tmp/hadoop-root-secondarynamenode.pid file is empty before retry.


In [20]:
#ssh_install()

#### Start yarn

In [21]:
# 11.1 Start yarn
! start-yarn.sh

Starting resourcemanager
resourcemanager is running as process 39969.  Stop it first and ensure /tmp/hadoop-root-resourcemanager.pid file is empty before retry.
Starting nodemanagers
localhost: nodemanager is running as process 6341.  Stop it first and ensure /tmp/hadoop-root-nodemanager.pid file is empty before retry.


If `start-dfs.sh` fails, issue the following three commands, one after another:<br>  
`! sudo apt-get remove openssh-client openssh-server`<br>
`! sudo apt-get install openssh-client openssh-server`<br>
`! service ssh restart`<br>

And then try to start hadoop again, as: `start-dfs.sh`

In [22]:
# If in safemode leave it
!hdfs dfsadmin -safemode leave

Safe mode is OFF


### Create folders in hadoop
Folders created are:
> /user <br>
> /user/ashok

 A file `airports.csv` is also created and deposited in /user/ashok 


In [23]:
# 11.1
print("\n---26. Make folders in hadoop----\n")
! hdfs dfs -mkdir /user
! hdfs dfs -mkdir /user/ashok


---26. Make folders in hadoop----

mkdir: `/user': File exists
mkdir: `/user/ashok': File exists


In [24]:
# 11.1.1 Create a csv file 
#          and tranfer it to hdfs:

!echo "a,b,c,d"   > /content/airports.csv
!echo "5,4,6,7"   >> /content/airports.csv
!echo "2,3,4,5"   >> /content/airports.csv
!echo "8,9,0,1"   >> /content/airports.csv
!echo "2,3,4,1"   >> /content/airports.csv
!echo "1,2,2,1"   >> /content/airports.csv
!echo "0,1,2,6"   >> /content/airports.csv
!echo "9,3,1,8"   >> /content/airports.csv


# 4.4
!hdfs dfs -rm -f /user/ashok/airports.csv
!hdfs dfs -put /content/airports.csv  /user/ashok/


## Check if folders created
Folders are:
>> /user <br>
>> /user/ashok

In [25]:
# 11.2 Run hadoop commands
#  If hadoop in safemode, execute
# !hdfs dfsadmin -safemode leave


! hdfs dfs -ls /
! hdfs dfs -ls /user
! hdfs dfs -ls /user/ashok

Found 1 items
drwxr-xr-x   - root supergroup          0 2023-03-27 07:37 /user
Found 1 items
drwxr-xr-x   - root supergroup          0 2023-03-27 07:41 /user/ashok
Found 1 items
-rw-r--r--   1 root supergroup         64 2023-03-27 07:41 /user/ashok/airports.csv


In [26]:
# 11.3 Stopping hadoop
#      Gives some errors
#      But hadoop stops
#!stop-dfs.sh

Run the `ssh_install()` again if hadoop fails to start with `start-dfs.sh` and then try to start hadoop again.

## Install spark

### Define functions

`findspark`: PySpark isn't on `sys.path` by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding `pyspark` to `sys.path` at runtime. `findspark` does the latter.

In [48]:
# 1.0 Function to download and unzip spark
def spark_koalas_install():
  print("\n--1.1 Install findspark----\n")
  !pip install -q findspark

  #print("\n--1.2 Install databricks Koalas----\n")
  #!pip install koalas

  print("\n--1.3 Download Apache tar.gz----\n")
  
  ! wget -c https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

  print("\n--1.4 Transfer downloaded content and unzip tar.gz----\n")
  !  mv /content/spark*   /opt/
  ! tar -xzf /opt/spark-3.3.2-bin-hadoop3.tgz  --directory /opt/

  print("\n--1.5 Check folder for files----\n")
  ! ls -la /opt


In [49]:
# 1.1 Function to set environment
def set_spark_env():
  print("\n---2. Set Environment variables----\n")
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre" 
  os.environ["SPARK_HOME"] = "/opt/spark-3.3.2-bin-hadoop3"     
  os.environ["LD_LIBRARY_PATH"] += ":/opt/spark-3.3.2-bin-hadoop3/lib/native"
  os.environ["PATH"] += ":/opt/spark-3.3.2-bin-hadoop3/bin:/opt/spark-3.3.2-bin-hadoop3/sbin"
  print("\n---2.1. Check Environment variables----\n")
  # Check
  ! echo $PATH
  ! echo $LD_LIBRARY_PATH

In [50]:
# 1.2 Function to configure spark 
def spark_conf():
  
  print("\n---3. Configure spark to access hadoop----\n")
  !mv /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh.template  /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh
  !echo "HADOOP_CONF_DIR=/opt/hadoop-3.3.5/etc/hadoop/" >> /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh
  print("\n---3. Done....----\n")
  # Amend properties
  # Create spark-defaults.conf
  print("\n---3.1 Increase spark memory----\n") 
  ! cp /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf.template  /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf
  ! echo "spark.driver.memory 6g" >> /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf
  ! echo "spark.executor.memory 3g" >> /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf
  print("\n---3.1 Done....----\n")
  #!cat /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh

### Install spark

In [51]:
# 2.0 Call all the three functions
def install_spark():
  spark_koalas_install()
  set_spark_env()
  spark_conf()


In [52]:
# 2.1 
# Delete earlier key store
install_spark()


--1.1 Install findspark----


--1.3 Download Apache tar.gz----

--2023-03-27 08:15:42--  https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299360284 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.2-bin-hadoop3.tgz’

spark-3.3.2-bin-had 100%[===================>] 285.49M   143MB/s    in 2.0s    

2023-03-27 08:15:44 (143 MB/s) - ‘spark-3.3.2-bin-hadoop3.tgz’ saved [299360284/299360284]


--1.4 Transfer downloaded content and unzip tar.gz----


--1.5 Check folder for files----

total 982352
drwxr-xr-x  1 root root      4096 Mar 27 08:15 .
drwxr-xr-x  1 root root      4096 Mar 27 05:54 ..
drwxr-xr-x  1 root root      4096 Mar 23 19:09 google
drwxr-xr-x 11 2002 2002      4096 Mar 15 16:58 hadoop-3.3.5
-rw-r--r--  1 root root 706533213 Mar 15 19:35 hado

## Call libraries
Hadoop should have been started<br>
Instead of `pandas`, `pyspark pandas` imported as `ps`


Call some libraries

In [53]:
# 3.0 Just call some libraries to test
import numpy as np

# 3.1 Get spark in sys.path
import findspark
findspark.init()

# 3.2 Call other spark libraries
#     Just to test
from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

## Create Spark session
Access spark as `spark`

In [54]:
# 3.1 Build spark session
spark = SparkSession. \
                    builder. \
                    master("local[*]"). \
                    appName("myexpt"). \
                    getOrCreate()


## Test spark

In [55]:
# 4.0 pyspark pandas DataFrame
pdf = ps.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c', 'd','d'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach','apple','orange'],
        'x3': [1, 1, 2, 2, 2, 4, 1, 2],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5, 3.0, 2.0],
        'y1': [1, 0, 1, 0, 0, 1, 1, 0],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes']
    })

# 4.1
pdf

,x1,x2,x3,x4,y1,y2
0,a,apple,1,2.4,1,yes
1,a,orange,1,2.5,0,no
2,b,orange,2,3.5,1,no
3,b,orange,2,1.4,0,yes
4,b,peach,2,2.1,0,yes
5,c,peach,4,1.5,1,yes
6,d,apple,1,3.0,1,no
7,d,orange,2,2.0,0,yes


In [56]:
# 4.2 Transform pyspark pandas DataFrame 
#       to SparkSQL DataFrame:

df = pdf.to_spark()
df.show()

/opt/spark-3.3.2-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
|  b| peach|  2|2.1|  0|yes|
|  c| peach|  4|1.5|  1|yes|
|  d| apple|  1|3.0|  1| no|
|  d|orange|  2|2.0|  0|yes|
+---+------+---+---+---+---+



In [36]:
# 4.3 Create a csv file 
#     and tranfer it to hdfs

!echo "a,b,c,d"   > /content/airports.csv
!echo "5,4,6,7"   >> /content/airports.csv
!echo "2,3,4,5"   >> /content/airports.csv
!echo "8,9,0,1"   >> /content/airports.csv
!echo "2,3,4,1"   >> /content/airports.csv
!echo "1,2,2,1"   >> /content/airports.csv
!echo "0,1,2,6"   >> /content/airports.csv
!echo "9,3,1,8"   >> /content/airports.csv
!ls -la /content

# 4.4
!hdfs dfs -rm -f /user/ashok/airports.csv
!hdfs dfs -put /content/airports.csv  /user/ashok/
!hdfs dfs -ls /user/ashok

total 24
drwxr-xr-x 1 root root 4096 Mar 27 07:41 .
drwxr-xr-x 1 root root 4096 Mar 27 05:54 ..
-rw-r--r-- 1 root root   64 Mar 27 07:42 airports.csv
drwxr-xr-x 4 root root 4096 Mar 23 18:56 .config
drwxr-xr-x 1 root root 4096 Mar 23 18:57 sample_data
-rwxr-xr-x 1 root root   92 Mar 27 07:15 xyz.sh
Deleted /user/ashok/airports.csv
Found 1 items
-rw-r--r--   1 root supergroup         64 2023-03-27 07:42 /user/ashok/airports.csv


In [37]:
# 5.0 Read file directly from hadoop
#      using sparkSQL  dataframe:

airports_df = spark.read.csv( 
                              "/user/ashok/airports.csv",
                              inferSchema = True,
                              header = True
                             )

# 5.1 Show file
airports_df.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  5|  4|  6|  7|
|  2|  3|  4|  5|
|  8|  9|  0|  1|
|  2|  3|  4|  1|
|  1|  2|  2|  1|
|  0|  1|  2|  6|
|  9|  3|  1|  8|
+---+---+---+---+



## Mount gdrive
And begin your work

In [57]:
# Mount gdrive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Your code here
Call machine learning libraries as needed

In [58]:
# Begin writing your code

In [ ]:
###############################